In [ ]:
'''
Catboost와 Lightgbm만 실험
'''

# 패키지 불러오기

In [14]:
import math

import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb

from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


n_splits = 5

# 모델 정의
 1. Lightgbm
 2. Catboost

In [15]:
def Xy_split(dataset):
    
    X = dataset.drop('연봉', axis=1)
    column_dict = {f'variable{idx+1}':col for idx, col in enumerate(X.columns)}
    X = X.rename(columns = {col:f'variable{idx+1}' for idx, col in enumerate(X.columns)})
    y = dataset['연봉']
    
    return column_dict, X, y

In [16]:
def lgbm_model(data_x, data_y):

    # lgbm 모델링
    lgb_params = {"objective" : "rmse",
                 "verbosity" : -1}

    performance = []

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :].values
        y_train = data_y[tr_idx].values

        X_valid = data_x.iloc[val_idx, :].values
        y_valid = data_y[val_idx].values

        lgb_dtrain = lgb.Dataset(data = X_train, label = y_train) 
        lgb_dvalid = lgb.Dataset(data = X_valid, label = y_valid) 

        lgb_model = lgb.train(lgb_params, lgb_dtrain, 20000, valid_sets=[lgb_dvalid], early_stopping_rounds=100, verbose_eval=1000)
        pred_valid = list(map(lambda x: 10 ** x, lgb_model.predict(X_valid)))
        rmse = mean_squared_error(list(map(lambda x: 10 ** x, y_valid)), pred_valid, squared=False)

        performance.append(rmse)

    # lgbm 성능종합
    performance.append(np.mean(performance))

    output = pd.DataFrame({'lgbm':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

In [17]:
def cb_model(data_x, data_y):

    # cb 모델링
    cat_cols = []
    performance = []

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        cb_dtrain = Pool(data=X_train, label=y_train, cat_features=cat_cols)
        cb_dvalid = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

        cb_model = CatBoostRegressor(iterations=20000, eval_metric='RMSE', loss_function='RMSE', verbose = 0)

        cb_model.fit(cb_dtrain, eval_set=cb_dvalid, early_stopping_rounds=100, verbose_eval=1000, use_best_model=True)
        pred_valid = list(map(lambda x: 10 ** x, cb_model.predict(X_valid)))
        rmse = mean_squared_error(list(map(lambda x: 10 ** x, y_valid)), pred_valid, squared=False)

        performance.append(rmse)
        
    # cb 성능종합
    performance.append(np.mean(performance))

    output = pd.DataFrame({'cb':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

# train, test분할 후 모델링

In [18]:
hitter = pd.read_csv('../선수데이터(전처리완료)/모델링용ver3/타자(모델링용_원핫인코딩)_train_ver3.csv')
pitcher = pd.read_csv('../선수데이터(전처리완료)/모델링용ver3/투수(모델링용_원핫인코딩)_train_ver3.csv')

#### 1. 타자

In [19]:
# train, test 분할
col_dict, X, y = Xy_split(hitter)

X = X.reset_index(drop=True)
y = pd.Series(list(map(lambda x: math.log10(x), y)))

In [9]:
# 타자데이터 도출
hitter_performance = pd.concat([round(lgbm_model(X, y), 1),
                                 round(cb_model(X, y), 1)], axis=1)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[88]	valid_0's rmse: 0.154875
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[78]	valid_0's rmse: 0.152386
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[130]	valid_0's rmse: 0.142379
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[84]	valid_0's rmse: 0.147658
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[56]	valid_0's rmse: 0.150343
Learning rate set to 0.008427
0:	learn: 0.4801643	test: 0.4947901	best: 0.4947901 (0)	total: 4.43ms	remaining: 1m 28s
1000:	learn: 0.1271586	test: 0.1598367	best: 0.1598367 (1000)	total: 4.81s	remaining: 1m 31s
2000:	learn: 0.0989121	test: 0.1537228	best: 0.1537176 (1986)	total: 9.45s	remaining: 1m 25s
3000:	learn: 0.0789916	test: 0.1513803	best: 0.1513775 (2997

#### 2. 투수

In [10]:
# train, test 분할
col_dict, X, y = Xy_split(pitcher)

X = X.reset_index(drop=True)
y = pd.Series(list(map(lambda x: math.log10(x), y)))

In [11]:
# 투수데이터 도출
pitcher_performance = pd.concat([round(lgbm_model(X, y), 1),
                                 round(cb_model(X, y), 1)], axis=1)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[69]	valid_0's rmse: 0.159021
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[104]	valid_0's rmse: 0.171641
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[93]	valid_0's rmse: 0.167131
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[56]	valid_0's rmse: 0.159078
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[154]	valid_0's rmse: 0.190338
Learning rate set to 0.008028
0:	learn: 0.4238886	test: 0.3961434	best: 0.3961434 (0)	total: 4.05ms	remaining: 1m 20s
1000:	learn: 0.1404233	test: 0.1685901	best: 0.1685901 (1000)	total: 4.27s	remaining: 1m 21s
2000:	learn: 0.1049120	test: 0.1634001	best: 0.1633826 (1986)	total: 8.49s	remaining: 1m 16s
3000:	learn: 0.0814293	test: 0.1617937	best: 0.1617906 (299

In [12]:
hitter_performance

,lgbm,cb
cv1,18477.4,18482.6
cv2,20707.2,20064.3
cv3,16578.2,17745.7
cv4,19689.7,19664.6
cv5,16540.1,16745.7
평균,18398.5,18540.6


In [13]:
pitcher_performance

,lgbm,cb
cv1,14682.1,15595.3
cv2,12763.6,13165.7
cv3,18535.7,19555.2
cv4,13645.6,14986.6
cv5,18906.4,19588.2
평균,15706.7,16578.2
